In [5]:
from pathlib import Path
import sys
from dotenv import load_dotenv


def find_repo_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / "SQL").is_dir():
            return p
    return start


repo_root = find_repo_root(Path.cwd())

if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

sql_env = repo_root / "SQL" / ".env"
root_env = repo_root / ".env"
dotenv_path = sql_env if sql_env.exists() else (root_env if root_env.exists() else None)
if dotenv_path:
    load_dotenv(dotenv_path=dotenv_path)
else:
    load_dotenv()

from SQL.snowflake_client import SnowflakeClient

# Create a reusable client for the session
if 'client' in globals():
    try:
        client.close()
    except Exception:
        pass
client = SnowflakeClient.from_env()

In [6]:
# Sample: select from calls with triple-quoted SQL
query = """
SELECT * FROM cases
LIMIT 100
"""

df = client.query_to_df(query)
df.head(100)

,CASE_ID,CREATED_DATE,CLOSED_DATE,CASE_ORIGIN,NUMBER_OF_EMAILS_RECEIVED,NUMBER_OF_EMAILS_SENT,CONTACT_REASON,BUCKET,TOPIC,SUB_TOPIC
0,500dV00000Dt3ApQAJ,2025-08-04 04:33:14,NaT,Backoffice,NaN,2.0,Systemic in Nature,Account Opening,Account Opening - MISC,None
1,500dV00000DtsgGQAR,2025-08-04 08:11:24,2025-08-04 12:32:53,Phone,NaN,2.0,System Issue,Account Management,Account Management - 2FA,None
2,500dV00000DtsgIQAR,2025-08-04 08:24:10,2025-08-04 14:17:02,Form_Mobile,1.0,1.0,Product Question,Corporate Actions,Corporate Actions - Stock Split,Securities Statement
3,500dV00000DttXXQAZ,2025-08-04 00:48:01,2025-08-04 14:17:02,Form_Mobile,1.0,1.0,Product Question,Corporate Actions,Corporate Actions - Stock Split,None
4,500dV00000Dtv80QAB,2025-08-04 00:51:30,2025-08-04 14:00:30,Form_Mobile,1.0,1.0,Product Question,Corporate Actions,Corporate Actions - Stock Split,None
...,...,...,...,...,...,...,...,...,...,...
95,500dV00000DuQVAQA3,2025-08-04 09:13:37,NaT,Form_Mobile,1.0,NaN,Null,Corporate Actions,Corporate Actions - Stock Split,None
96,500dV00000DuQrZQAV,2025-08-04 07:14:34,2025-08-04 14:18:58,Form_Mobile,1.0,1.0,Product Question,Corporate Actions,Corporate Actions - Stock Split,None
97,500dV00000DuQtPQAV,2025-08-04 08:59:47,2025-08-05 09:08:29,Form_Mobile,1.0,1.0,Product Question,Payments,Payments - Withdrawals,None
98,500dV00000DuR2xQAF,2025-08-04 09:39:52,2025-08-11 11:50:30,Form_Mobile,1.0,1.0,New Scalable,Broker Trading,Broker Trading - Savings Plans,None


In [9]:
# Sample: select from calls with triple-quoted SQL
query = """
WITH anchor AS (
  SELECT TO_TIMESTAMP_NTZ('2025-08-13 10:06:21') AS window_end
),
enriched AS (
  SELECT
    c.*,
    -- Open as of anchor (handles future/NULL closures defensively)
    IFF(c.CLOSED_DATE IS NULL OR c.CLOSED_DATE > (SELECT window_end FROM anchor), 1, 0) AS is_open_asof,
    -- Age measured to CLOSED_DATE (if before anchor) or to anchor (if open/after)
    DATEDIFF(
      'hour',
      c.CREATED_DATE,
      LEAST(COALESCE(c.CLOSED_DATE, (SELECT window_end FROM anchor)), (SELECT window_end FROM anchor))
    ) AS age_hours_asof
  FROM SCALABLECAPITAL.PUBLIC.CASES c
)
SELECT
  BUCKET,
  TOPIC,
  COUNT_IF(is_open_asof = 1)                               AS open_now,
  COUNT_IF(is_open_asof = 1 AND age_hours_asof >= 24)      AS open_gt_1d,
  COUNT_IF(is_open_asof = 1 AND age_hours_asof >= 72)      AS open_gt_3d,
  COUNT_IF(is_open_asof = 1 AND age_hours_asof >= 168)     AS open_gt_7d
FROM enriched
GROUP BY 1,2
ORDER BY open_now DESC, BUCKET, TOPIC;;


"""

df = client.query_to_df(query)
df.head(100)

,BUCKET,TOPIC,OPEN_NOW,OPEN_GT_1D,OPEN_GT_3D,OPEN_GT_7D
0,Broker Trading,Broker Trading - MISC,253,253,243,114
1,Broker Portfolio Management,Broker Portfolio Management - Transfer IN,193,193,188,95
2,Account Management,Account Management - Data Change,178,178,161,67
3,Broker Trading,Broker Trading - Savings Plans,142,142,140,73
4,Account Opening,Account Opening - MISC,138,138,124,52
5,Taxes,Taxes - MISC,133,133,130,54
6,Null,Null,115,115,112,52
7,Broker Portfolio Management,Broker Portfolio Management - MISC,113,113,109,40
8,Broker Trading,Broker Trading - Trade Issues,91,91,90,44
9,Corporate Actions,Corporate Actions - Stock Split,84,84,84,59


In [27]:
# Sample: select from calls with triple-quoted SQL
query = """
WITH anchor AS (
  SELECT TO_TIMESTAMP_NTZ('2025-08-13 10:06:21') AS window_end
),
base AS (
  SELECT
    CASE_ID,
    CREATED_DATE,
    CLOSED_DATE,
    CASE_ORIGIN,
    NUMBER_OF_EMAILS_RECEIVED,
    NUMBER_OF_EMAILS_SENT,
    CONTACT_REASON,
    BUCKET,
    TOPIC,
    SUB_TOPIC
  FROM SCALABLECAPITAL.PUBLIC.CASES
),
enriched AS (
  SELECT
    b.*,
    IFF(b.CLOSED_DATE IS NULL OR b.CLOSED_DATE > (SELECT window_end FROM anchor), 1, 0) AS is_open_asof,
    DATEDIFF(
      'hour',
      b.CREATED_DATE,
      LEAST(COALESCE(b.CLOSED_DATE, (SELECT window_end FROM anchor)), (SELECT window_end FROM anchor))
    ) AS age_hours_asof,
    DATEDIFF('hour', b.CREATED_DATE, (SELECT window_end FROM anchor)) AS since_creation_to_anchor_hours,
    IFF(DATEDIFF('hour', b.CREATED_DATE, (SELECT window_end FROM anchor)) >=  24, 1, 0) AS elig_1d,
    IFF(DATEDIFF('hour', b.CREATED_DATE, (SELECT window_end FROM anchor)) >=  72, 1, 0) AS elig_3d,
    IFF(DATEDIFF('hour', b.CREATED_DATE, (SELECT window_end FROM anchor)) >= 168, 1, 0) AS elig_7d
  FROM base b
)
SELECT
  BUCKET,
  CAST(COUNT(*) AS NUMBER(38)) AS ORIGINATED_CASES,
  CAST(COUNT_IF(is_open_asof = 1) AS NUMBER(38)) AS OPEN_NOW,
  CAST(ROUND(COUNT_IF(is_open_asof = 1) / NULLIF(COUNT(*), 0), 2) AS NUMBER(38,2)) AS OPEN_NOW_RATE,
  CAST(ROUND(COUNT_IF(is_open_asof = 1 AND age_hours_asof >= 24)  / NULLIF(SUM(elig_1d), 0), 2) AS NUMBER(38,2)) AS OPEN_1D_RATE,
  CAST(ROUND(COUNT_IF(is_open_asof = 1 AND age_hours_asof >= 72)  / NULLIF(SUM(elig_3d), 0), 2) AS NUMBER(38,2)) AS OPEN_3D_RATE,
  CAST(ROUND(COUNT_IF(is_open_asof = 1 AND age_hours_asof >= 168) / NULLIF(SUM(elig_7d), 0), 2) AS NUMBER(38,2)) AS OPEN_7D_RATE
FROM enriched
GROUP BY 1
ORDER BY OPEN_NOW_RATE DESC, BUCKET;


"""

df = client.query_to_df(query)
df.head(100)

,BUCKET,ORIGINATED_CASES,OPEN_NOW,OPEN_NOW_RATE,OPEN_1D_RATE,OPEN_3D_RATE,OPEN_7D_RATE
0,Null,116,115,0.99,0.99,0.99,1.00
1,Taxes,463,352,0.76,0.76,0.75,0.75
2,Broker Trading,929,505,0.54,0.54,0.54,0.60
3,Broker Portfolio Management,773,359,0.46,0.46,0.46,0.46
4,Corporate Actions,545,185,0.34,0.34,0.34,0.27
5,Account Opening,758,241,0.32,0.32,0.31,0.33
6,Goodwill,112,30,0.27,0.27,0.26,0.16
7,Account Management,1759,420,0.24,0.24,0.23,0.20
8,Payments,717,105,0.15,0.15,0.14,0.06
9,Other,20,0,0.00,0.00,0.00,0.00


In [40]:
# Sample: select from calls with triple-quoted SQL
query = """
WITH anchor AS (
  SELECT TO_TIMESTAMP_NTZ('2025-08-13 10:06:21') AS window_end
),
base AS (
  SELECT
    CASE_ID, CREATED_DATE, CLOSED_DATE, CASE_ORIGIN,
    NUMBER_OF_EMAILS_RECEIVED, NUMBER_OF_EMAILS_SENT,
    CONTACT_REASON, BUCKET, TOPIC, SUB_TOPIC
  FROM SCALABLECAPITAL.PUBLIC.CASES
  WHERE BUCKET IN ('Taxes','Broker Trading','Broker Portfolio Management')
),
enriched AS (
  SELECT
    b.*,
    IFF(b.CLOSED_DATE IS NULL OR b.CLOSED_DATE > (SELECT window_end FROM anchor), 1, 0) AS is_open_asof,
    DATEDIFF('hour', b.CREATED_DATE,
      LEAST(COALESCE(b.CLOSED_DATE, (SELECT window_end FROM anchor)), (SELECT window_end FROM anchor))
    ) AS age_hours_asof,
    IFF(DATEDIFF('hour', b.CREATED_DATE, (SELECT window_end FROM anchor)) >=  72, 1, 0) AS elig_3d,
    IFF(DATEDIFF('hour', b.CREATED_DATE, (SELECT window_end FROM anchor)) >= 168, 1, 0) AS elig_7d
  FROM base b
)
SELECT
  BUCKET,
  TOPIC,
  CAST(COUNT(*) AS NUMBER(38)) AS originated_cases,
  CAST(COUNT_IF(is_open_asof = 1) AS NUMBER(38)) AS open_now,
  CAST(ROUND(COUNT_IF(is_open_asof = 1) / NULLIF(COUNT(*), 0), 2) AS NUMBER(38,2))                    AS open_now_rate,

  CAST(ROUND(COUNT_IF(is_open_asof = 1 AND age_hours_asof >= 168) / NULLIF(SUM(elig_7d), 0), 2)
       AS NUMBER(38,2)) AS open_7d_rate
FROM enriched
GROUP BY 1,2
ORDER BY bucket, open_now desc, open_7d_rate desc;


"""

df = client.query_to_df(query)
df.head(100)

,BUCKET,TOPIC,ORIGINATED_CASES,OPEN_NOW,OPEN_NOW_RATE,OPEN_7D_RATE
0,Broker Portfolio Management,Broker Portfolio Management - Transfer IN,291,193,0.66,0.68
1,Broker Portfolio Management,Broker Portfolio Management - MISC,135,113,0.84,0.78
2,Broker Portfolio Management,Broker Portfolio Management - Transfer OUT,318,36,0.11,0.10
3,Broker Portfolio Management,Broker Portfolio Management - Broker Functions,29,17,0.59,0.62
4,Broker Trading,Broker Trading - MISC,298,253,0.85,0.84
5,Broker Trading,Broker Trading - Savings Plans,346,142,0.41,0.43
6,Broker Trading,Broker Trading - Trade Issues,174,91,0.52,0.72
7,Broker Trading,Broker Trading - Derivatives,19,7,0.37,0.71
8,Broker Trading,Broker Trading - Crypto,19,5,0.26,0.50
9,Broker Trading,Broker Trading - EIX Support,3,3,1.00,None


In [51]:
# Sample: select from calls with triple-quoted SQL
query = """
WITH anchor AS (
  SELECT TO_TIMESTAMP_NTZ('2025-08-13 10:06:21') AS window_end
),
base AS (
  SELECT
    CASE_ID,
    CREATED_DATE,
    CLOSED_DATE,
    CASE_ORIGIN,
    NUMBER_OF_EMAILS_RECEIVED,
    NUMBER_OF_EMAILS_SENT,
    CONTACT_REASON,
    BUCKET,
    TOPIC,
    SUB_TOPIC
  FROM SCALABLECAPITAL.PUBLIC.CASES
  WHERE BUCKET IN ('Taxes','Broker Trading','Broker Portfolio Management')
),
enriched AS (
  SELECT
    b.*,
    IFF(b.CLOSED_DATE IS NULL OR b.CLOSED_DATE > (SELECT window_end FROM anchor), 1, 0) AS is_open_asof,
    DATEDIFF(
      'hour',
      b.CREATED_DATE,
      LEAST(COALESCE(b.CLOSED_DATE, (SELECT window_end FROM anchor)), (SELECT window_end FROM anchor))
    ) AS age_hours_asof,
    IFF(DATEDIFF('hour', b.CREATED_DATE, (SELECT window_end FROM anchor)) >=  72, 1, 0) AS elig_3d,
    IFF(DATEDIFF('hour', b.CREATED_DATE, (SELECT window_end FROM anchor)) >= 168, 1, 0) AS elig_7d
  FROM base b
)
SELECT
  BUCKET,
  TOPIC,
  CAST(COUNT(*) AS NUMBER(38)) AS ORIGINATED_CASES,
  CAST(COUNT_IF(is_open_asof = 1) AS NUMBER(38)) AS OPEN_NOW,
  CAST(ROUND(COUNT_IF(is_open_asof = 1) / NULLIF(COUNT(*), 0), 2) AS NUMBER(38,2)) AS OPEN_NOW_RATE,
  CAST(ROUND(COUNT_IF(is_open_asof = 1 AND age_hours_asof >= 168) / NULLIF(SUM(elig_7d), 0), 2) AS NUMBER(38,2)) AS OPEN_7D_RATE,
  CAST(
    ROUND(
      COUNT_IF(CONTACT_REASON = 'Null') / NULLIF(COUNT(*), 0),
      2
    ) AS NUMBER(38,2)
  ) AS PCT_CONTACT_REASON_NULL
FROM enriched
GROUP BY 1,2
ORDER BY BUCKET, PCT_CONTACT_REASON_NULL desc;

"""

df = client.query_to_df(query)
pd.set_option('display.max_rows', None)  
df

,BUCKET,TOPIC,ORIGINATED_CASES,OPEN_NOW,OPEN_NOW_RATE,OPEN_7D_RATE,PCT_CONTACT_REASON_NULL
0,Broker Portfolio Management,Broker Portfolio Management - MISC,135,113,0.84,0.78,0.73
1,Broker Portfolio Management,Broker Portfolio Management - Transfer IN,291,193,0.66,0.68,0.53
2,Broker Portfolio Management,Broker Portfolio Management - Broker Functions,29,17,0.59,0.62,0.45
3,Broker Portfolio Management,Broker Portfolio Management - Transfer OUT,318,36,0.11,0.10,0.16
4,Broker Trading,Broker Trading - MISC,298,253,0.85,0.84,0.81
5,Broker Trading,Broker Trading - EIX Support,3,3,1.00,None,0.67
6,Broker Trading,Broker Trading - Trade Issues,174,91,0.52,0.72,0.52
7,Broker Trading,Broker Trading - Savings Plans,346,142,0.41,0.43,0.28
8,Broker Trading,Broker Trading - Scalable Products,24,0,0.00,0.00,0.21
9,Broker Trading,Broker Trading - Derivatives,19,7,0.37,0.71,0.21


In [54]:
# Sample: select from calls with triple-quoted SQL
query = """
WITH anchor AS (
  -- We still define the snapshot end because we will calculate observed_age_hours
  -- but for this speed metric we only keep actually closed cases
  SELECT TO_TIMESTAMP_NTZ('2025-08-13 10:06:21') AS window_end
),
base AS (
  SELECT
    CASE_ID,
    CREATED_DATE,
    CLOSED_DATE,
    CASE_ORIGIN,
    NUMBER_OF_EMAILS_RECEIVED,
    NUMBER_OF_EMAILS_SENT,
    CONTACT_REASON,
    BUCKET,
    TOPIC,
    SUB_TOPIC
  FROM SCALABLECAPITAL.PUBLIC.CASES
),
enriched AS (
  SELECT
    b.*,
    -- how long this case took from create to close, in hours
    DATEDIFF(
      'hour',
      b.CREATED_DATE,
      b.CLOSED_DATE
    ) AS ttc_hours,  -- time to close

    -- whether it closed the same calendar day it was opened
    IFF(
      DATE_TRUNC('day', b.CREATED_DATE) = DATE_TRUNC('day', b.CLOSED_DATE),
      1, 0
    ) AS same_day_flag
  FROM base b
  WHERE b.CLOSED_DATE IS NOT NULL
)
SELECT
  BUCKET,
  TOPIC,
  CAST(COUNT(*) AS NUMBER(38))                                        AS closed_cases,
  CAST(ROUND(MEDIAN(ttc_hours), 2) AS NUMBER(38,2))                    AS median_ttc_hours,
  CAST(ROUND(APPROX_PERCENTILE(ttc_hours, 0.90), 2) AS NUMBER(38,2))   AS p90_ttc_hours,
  CAST(ROUND(AVG(same_day_flag), 2) AS NUMBER(38,2))                   AS same_day_close_rate
FROM enriched
GROUP BY 1,2
HAVING closed_cases >= 10
ORDER BY bucket, topic, median_ttc_hours DESC;

"""

df = client.query_to_df(query)
pd.set_option('display.max_rows', None)  
df

,BUCKET,TOPIC,CLOSED_CASES,MEDIAN_TTC_HOURS,P90_TTC_HOURS,SAME_DAY_CLOSE_RATE
0,Account Management,Account Management - 2FA,154,23.00,79.40,0.38
1,Account Management,Account Management - Cancellation,183,24.00,86.00,0.19
2,Account Management,Account Management - Client Documents,285,23.00,73.60,0.27
3,Account Management,Account Management - Data Change,154,25.00,167.00,0.11
4,Account Management,Account Management - IT Issues,136,23.00,83.00,0.22
5,Account Management,Account Management - MISC,415,23.00,74.00,0.41
6,Account Opening,Account Opening - Account Reactivation,167,0.00,71.40,0.66
7,Account Opening,Account Opening - KYC,116,65.50,140.50,0.11
8,Account Opening,Account Opening - MISC,58,51.00,94.00,0.29
9,Account Opening,Account Opening - Opening Process,176,1.00,72.50,0.60


In [55]:
# Sample: select from calls with triple-quoted SQL
query = """
WITH anchor AS (
  SELECT TO_TIMESTAMP_NTZ('2025-08-13 10:06:21') AS window_end
),
base AS (
  SELECT
    CASE_ID, CREATED_DATE, CLOSED_DATE, CASE_ORIGIN,
    NUMBER_OF_EMAILS_RECEIVED, NUMBER_OF_EMAILS_SENT,
    CONTACT_REASON, BUCKET, TOPIC, SUB_TOPIC
  FROM SCALABLECAPITAL.PUBLIC.CASES
),
enriched AS (
  SELECT
    b.*,
    IFF(b.CLOSED_DATE IS NULL OR b.CLOSED_DATE > (SELECT window_end FROM anchor), 1, 0) AS is_open_asof,
    DATEDIFF('hour', b.CREATED_DATE,
      LEAST(COALESCE(b.CLOSED_DATE,(SELECT window_end FROM anchor)),(SELECT window_end FROM anchor))
    ) AS age_hours_asof,
    IFF(DATEDIFF('hour', b.CREATED_DATE, (SELECT window_end FROM anchor)) >= 168, 1, 0) AS elig_7d,
    COALESCE(NUMBER_OF_EMAILS_RECEIVED,0) + COALESCE(NUMBER_OF_EMAILS_SENT,0) AS emails_total,
    IFF(DATE_TRUNC('day', b.CREATED_DATE) = DATE_TRUNC('day', b.CLOSED_DATE), 1, 0) AS same_day_flag
  FROM base b
)
SELECT
  CASE_ORIGIN,
  CAST(COUNT(*) AS NUMBER(38))                                                        AS TOTAL_CASES,
  CAST(ROUND(COUNT(*) / NULLIF(SUM(COUNT(*)) OVER (), 0), 2) AS NUMBER(38,2))        AS SHARE_OF_TOTAL,
  CAST(COUNT_IF(is_open_asof = 1) AS NUMBER(38))                                      AS OPEN_NOW,
  CAST(ROUND(COUNT_IF(is_open_asof = 1) / NULLIF(COUNT(*), 0), 2) AS NUMBER(38,2))   AS OPEN_NOW_RATE,
  CAST(ROUND(COUNT_IF(is_open_asof = 1 AND age_hours_asof >= 168) /
             NULLIF(SUM(elig_7d), 0), 2) AS NUMBER(38,2))                             AS OPEN_7D_RATE,
  CAST(COUNT_IF(is_open_asof = 0) AS NUMBER(38))                                      AS CLOSED_CASES,
  CAST(ROUND(MEDIAN(IFF(is_open_asof=0, age_hours_asof, NULL)), 2) AS NUMBER(38,2))  AS MEDIAN_TTC_HOURS,
  CAST(ROUND(APPROX_PERCENTILE(IFF(is_open_asof=0, age_hours_asof, NULL), 0.90), 2)
       AS NUMBER(38,2))                                                               AS P90_TTC_HOURS,
  CAST(ROUND(AVG(IFF(is_open_asof=0, same_day_flag, NULL)), 2) AS NUMBER(38,2))      AS SAME_DAY_CLOSE_RATE,
  CAST(ROUND(AVG(emails_total), 2) AS NUMBER(38,2))                                   AS AVG_EMAILS_TOTAL,
  CAST(ROUND(COUNT_IF(CONTACT_REASON = 'Null') / NULLIF(COUNT(*), 0), 2)
       AS NUMBER(38,2))                                                               AS PCT_CONTACT_REASON_NULL
FROM enriched
GROUP BY CASE_ORIGIN
ORDER BY SHARE_OF_TOTAL DESC, OPEN_NOW_RATE DESC;

"""

df = client.query_to_df(query)
pd.set_option('display.max_rows', None)  
df

,CASE_ORIGIN,TOTAL_CASES,SHARE_OF_TOTAL,OPEN_NOW,OPEN_NOW_RATE,OPEN_7D_RATE,CLOSED_CASES,MEDIAN_TTC_HOURS,P90_TTC_HOURS,SAME_DAY_CLOSE_RATE,AVG_EMAILS_TOTAL,PCT_CONTACT_REASON_NULL
0,Form_Mobile,2241,0.36,1030,0.46,0.40,1211,24.00,97.00,0.27,2.03,0.51
1,Phone,1180,0.19,197,0.17,0.15,983,0.00,23.80,0.87,0.90,0.00
2,Email,1030,0.17,368,0.36,0.31,662,24.00,88.70,0.20,2.46,0.31
3,Form_Web,850,0.14,416,0.49,0.45,434,23.00,97.00,0.21,1.98,0.56
4,Outbound,418,0.07,74,0.18,0.33,344,0.00,25.40,0.86,1.18,0.54
5,Post,282,0.05,74,0.26,0.28,208,24.00,115.30,0.15,2.34,0.28
6,Backoffice,187,0.03,153,0.82,0.66,34,93.00,160.40,0.00,2.37,0.01
7,OSKAR Email,1,0.00,0,0.00,None,1,96.00,96.00,0.00,2.00,0.00
8,Order Email,3,0.00,0,0.00,0.00,3,20.00,36.80,0.33,6.00,0.00


In [62]:
# Sample: select from calls with triple-quoted SQL
query = """
WITH base AS (
  SELECT
    EXTRACT(hour FROM CREATED_DATE) AS hour_of_day,
    CASE_ORIGIN
  FROM SCALABLECAPITAL.PUBLIC.CASES
  WHERE CREATED_DATE IS NOT NULL
),
per_hour AS (
  SELECT
    hour_of_day,
    COUNT(*) AS total_cases,
    SUM(IFF(CASE_ORIGIN = 'Form_Mobile', 1, 0)) AS c_form_mobile,
    SUM(IFF(CASE_ORIGIN = 'Form_Web',    1, 0)) AS c_form_web,
    SUM(IFF(CASE_ORIGIN = 'Phone',       1, 0)) AS c_phone,
    SUM(IFF(CASE_ORIGIN = 'Email',       1, 0)) AS c_email,
    SUM(IFF(CASE_ORIGIN = 'Outbound',    1, 0)) AS c_outbound,
    SUM(IFF(CASE_ORIGIN = 'Post',        1, 0)) AS c_post,
    SUM(IFF(CASE_ORIGIN = 'Backoffice',  1, 0)) AS c_backoffice,
    SUM(IFF(CASE_ORIGIN = 'OSKAR Email', 1, 0)) AS c_oskar_email,
    SUM(IFF(CASE_ORIGIN = 'Order Email', 1, 0)) AS c_order_email
  FROM base
  GROUP BY 1
)
SELECT
  hour_of_day,
  CAST(total_cases AS NUMBER(38)) AS total_cases,
  CAST(ROUND(c_form_mobile  / NULLIF(total_cases, 0), 2) AS NUMBER(38,2)) AS share_form_mobile,
  CAST(ROUND(c_form_web     / NULLIF(total_cases, 0), 2) AS NUMBER(38,2)) AS share_form_web,
  CAST(ROUND(c_phone        / NULLIF(total_cases, 0), 2) AS NUMBER(38,2)) AS share_phone,
  CAST(ROUND(c_email        / NULLIF(total_cases, 0), 2) AS NUMBER(38,2)) AS share_email,
  CAST(ROUND(c_outbound     / NULLIF(total_cases, 0), 2) AS NUMBER(38,2)) AS share_outbound,
  CAST(ROUND(c_post         / NULLIF(total_cases, 0), 2) AS NUMBER(38,2)) AS share_post,
  CAST(ROUND(c_backoffice   / NULLIF(total_cases, 0), 2) AS NUMBER(38,2)) AS share_backoffice,
  CAST(ROUND(c_oskar_email  / NULLIF(total_cases, 0), 2) AS NUMBER(38,2)) AS share_oskar_email,
  CAST(ROUND(c_order_email  / NULLIF(total_cases, 0), 2) AS NUMBER(38,2)) AS share_order_email
FROM per_hour
ORDER BY hour_of_day;
"""

df = client.query_to_df(query)
pd.set_option('display.max_rows', None)  
df

,HOUR_OF_DAY,TOTAL_CASES,SHARE_FORM_MOBILE,SHARE_FORM_WEB,SHARE_PHONE,SHARE_EMAIL,SHARE_OUTBOUND,SHARE_POST,SHARE_BACKOFFICE,SHARE_OSKAR_EMAIL,SHARE_ORDER_EMAIL
0,0,49,0.57,0.12,0.00,0.24,0.00,0.00,0.06,0.00,0.00
1,1,24,0.83,0.04,0.00,0.08,0.00,0.00,0.04,0.00,0.00
2,2,12,0.58,0.17,0.00,0.25,0.00,0.00,0.00,0.00,0.00
3,3,16,0.50,0.06,0.00,0.44,0.00,0.00,0.00,0.00,0.00
4,4,8,0.75,0.00,0.00,0.13,0.00,0.00,0.13,0.00,0.00
5,5,17,0.82,0.00,0.00,0.12,0.00,0.00,0.06,0.00,0.00
6,6,56,0.80,0.04,0.00,0.16,0.00,0.00,0.00,0.00,0.00
7,7,138,0.73,0.09,0.00,0.17,0.00,0.00,0.01,0.00,0.00
8,8,311,0.50,0.13,0.18,0.12,0.05,0.00,0.03,0.00,0.00
9,9,482,0.38,0.11,0.26,0.15,0.08,0.00,0.02,0.00,0.00


In [65]:
# Sample: select from calls with triple-quoted SQL
query = """
WITH anchor AS (
  SELECT TO_TIMESTAMP_NTZ('2025-08-13 10:06:21') AS window_end
),
base AS (
  SELECT
    CASE_ID,
    CREATED_DATE,
    CLOSED_DATE,
    CONTACT_REASON,
    BUCKET,
    TOPIC
  FROM SCALABLECAPITAL.PUBLIC.CASES
  WHERE CONTACT_REASON = 'System Issue'
),
enriched AS (
  SELECT
    b.*,
    /* is it still open as of the snapshot window_end? */
    IFF(
      b.CLOSED_DATE IS NULL OR b.CLOSED_DATE > (SELECT window_end FROM anchor),
      1, 0
    ) AS is_open_asof,
    /* how long it's been alive as of close/snapshot */
    DATEDIFF(
      'hour',
      b.CREATED_DATE,
      LEAST(
        COALESCE(b.CLOSED_DATE, (SELECT window_end FROM anchor)),
        (SELECT window_end FROM anchor)
      )
    ) AS age_hours_asof,
    /* eligible for 7d aging bucket */
    IFF(
      DATEDIFF('hour', b.CREATED_DATE, (SELECT window_end FROM anchor)) >= 168,
      1, 0
    ) AS elig_7d
  FROM base b
)
SELECT
  BUCKET,
  TOPIC,
  /* total system issue cases in this bucket/topic */
  CAST(COUNT(*) AS NUMBER(38)) AS SYSTEM_ISSUE_CASES,

  /* how many are still open right now */
  CAST(COUNT_IF(is_open_asof = 1) AS NUMBER(38)) AS OPEN_NOW,

  /* how "stuck" they are overall */
  CAST(ROUND(COUNT_IF(is_open_asof = 1) / NULLIF(COUNT(*), 0), 2)
       AS NUMBER(38,2)) AS OPEN_NOW_RATE
FROM enriched
GROUP BY BUCKET, TOPIC
ORDER BY SYSTEM_ISSUE_CASES DESC, OPEN_NOW_RATE DESC;
"""

df = client.query_to_df(query)
pd.set_option('display.max_rows', None)  
df

,BUCKET,TOPIC,SYSTEM_ISSUE_CASES,OPEN_NOW,OPEN_NOW_RATE
0,Account Management,Account Management - IT Issues,52,7,0.13
1,Broker Trading,Broker Trading - Downtime,16,2,0.13
2,Broker Trading,Broker Trading - Trade Issues,16,1,0.06
3,Account Management,Account Management - 2FA,13,3,0.23
4,Account Management,Account Management - Client Documents,11,0,0.00
5,Account Management,Account Management - MISC,8,1,0.13
6,Payments,Payments - Account Balance,8,0,0.00
7,Payments,Payments - Deposits,7,0,0.00
8,Broker Portfolio Management,Broker Portfolio Management - Transfer IN,3,3,1.00
9,Broker Portfolio Management,Broker Portfolio Management - Broker Functions,3,1,0.33


In [66]:
# Sample: select from calls with triple-quoted SQL
query = """
WITH base AS (
  SELECT CASE_ORIGIN, BUCKET
  FROM SCALABLECAPITAL.PUBLIC.CASES
),
enriched AS (
  SELECT
    CASE_ORIGIN,
    BUCKET,
    IFF(BUCKET IS NULL OR TRIM(BUCKET) = '' OR UPPER(TRIM(BUCKET)) = 'NULL', 1, 0) AS is_bucket_nullish
  FROM base
)
SELECT
  CASE_ORIGIN,
  COUNT(*) AS total_cases,
  COUNT_IF(is_bucket_nullish = 1) AS null_buckets,
  CAST(ROUND(COUNT_IF(is_bucket_nullish = 1) / NULLIF(COUNT(*),0), 2) AS NUMBER(38,2)) AS null_bucket_rate
FROM enriched
GROUP BY CASE_ORIGIN
ORDER BY null_buckets DESC, null_bucket_rate DESC;
"""

df = client.query_to_df(query)
pd.set_option('display.max_rows', None)  
df

,CASE_ORIGIN,TOTAL_CASES,NULL_BUCKETS,NULL_BUCKET_RATE
0,Email,1030,115,0.11
1,Form_Mobile,2241,1,0.00
2,Form_Web,850,0,0.00
3,Post,282,0,0.00
4,Outbound,418,0,0.00
5,OSKAR Email,1,0,0.00
6,Backoffice,187,0,0.00
7,Phone,1180,0,0.00
8,Order Email,3,0,0.00


In [68]:
# Sample: select from calls with triple-quoted SQL
query = """
WITH base AS (
  SELECT BUCKET, CONTACT_REASON
  FROM SCALABLECAPITAL.PUBLIC.CASES
),
enriched AS (
  SELECT
    COALESCE(BUCKET, '(NULL)') AS bucket_group,
    IFF(CONTACT_REASON IS NULL OR TRIM(CONTACT_REASON) = '' OR UPPER(TRIM(CONTACT_REASON)) = 'NULL', 1, 0) AS is_cr_nullish
  FROM base
)
SELECT
  bucket_group AS bucket,
  COUNT(*) AS total_cases,
  COUNT_IF(is_cr_nullish = 1) AS null_contact_reasons,
  CAST(ROUND(COUNT_IF(is_cr_nullish = 1) / NULLIF(COUNT(*),0), 2) AS NUMBER(38,2)) AS null_contact_reason_rate
FROM enriched
GROUP BY bucket_group
ORDER BY null_contact_reason_rate DESC;
"""

df = client.query_to_df(query)
pd.set_option('display.max_rows', None)  
df

,BUCKET,TOTAL_CASES,NULL_CONTACT_REASONS,NULL_CONTACT_REASON_RATE
0,Null,116,115,0.99
1,Taxes,463,332,0.72
2,Broker Trading,929,443,0.48
3,Broker Portfolio Management,773,315,0.41
4,Goodwill,112,35,0.31
5,Account Management,1759,520,0.30
6,Account Opening,758,187,0.25
7,Payments,717,175,0.24
8,Corporate Actions,545,115,0.21
9,Other,20,2,0.10


In [69]:
# Sample: select from calls with triple-quoted SQL
query = """
WITH base AS (
  SELECT BUCKET, TOPIC
  FROM SCALABLECAPITAL.PUBLIC.CASES
),
enriched AS (
  SELECT
    COALESCE(BUCKET, '(NULL)') AS bucket_group,
    IFF(TOPIC IS NULL OR TRIM(TOPIC) = '' OR UPPER(TRIM(TOPIC)) = 'NULL', 1, 0) AS is_topic_nullish
  FROM base
)
SELECT
  bucket_group AS bucket,
  COUNT(*) AS total_cases,
  COUNT_IF(is_topic_nullish = 1) AS null_topics,
  CAST(ROUND(COUNT_IF(is_topic_nullish = 1) / NULLIF(COUNT(*),0), 2) AS NUMBER(38,2)) AS null_topic_rate
FROM enriched
GROUP BY bucket_group
ORDER BY null_topics DESC, null_topic_rate DESC;
"""

df = client.query_to_df(query)
pd.set_option('display.max_rows', None)  
df

,BUCKET,TOTAL_CASES,NULL_TOPICS,NULL_TOPIC_RATE
0,Null,116,116,1.00
1,Broker Portfolio Management,773,0,0.00
2,Broker Trading,929,0,0.00
3,Taxes,463,0,0.00
4,Account Opening,758,0,0.00
5,Account Management,1759,0,0.00
6,Payments,717,0,0.00
7,Goodwill,112,0,0.00
8,Corporate Actions,545,0,0.00
9,Other,20,0,0.00


In [72]:
# Sample: select from calls with triple-quoted SQL
query = """
WITH base AS (
  SELECT TOPIC, SUB_TOPIC
  FROM SCALABLECAPITAL.PUBLIC.CASES
),
enriched AS (
  SELECT
    COALESCE(TOPIC, '(NULL)') AS topic_group,
    IFF(SUB_TOPIC IS NULL OR TRIM(SUB_TOPIC) = '' OR UPPER(TRIM(SUB_TOPIC)) = 'NULL', 1, 0) AS is_sub_topic_nullish
  FROM base
)
SELECT
  topic_group AS topic,
  COUNT(*) AS total_cases,
  COUNT_IF(is_sub_topic_nullish = 1) AS null_sub_topics,
  CAST(ROUND(COUNT_IF(is_sub_topic_nullish = 1) / NULLIF(COUNT(*),0), 2) AS NUMBER(38,2)) AS null_sub_topic_rate
FROM enriched
GROUP BY topic_group
ORDER BY null_sub_topic_rate DESC, total_cases DESC;
"""

df = client.query_to_df(query)
pd.set_option('display.max_rows', None)  
df

,TOPIC,TOTAL_CASES,NULL_SUB_TOPICS,NULL_SUB_TOPIC_RATE
0,Account Management - MISC,498,498,1.00
1,Corporate Actions - Stock Split,350,349,1.00
2,Broker Trading - Savings Plans,346,346,1.00
3,Broker Portfolio Management - Transfer OUT,318,318,1.00
4,Broker Trading - MISC,298,298,1.00
5,Broker Portfolio Management - Transfer IN,291,291,1.00
6,Payments - Deposits,249,249,1.00
7,Account Management - Cancellation,218,218,1.00
8,Payments - Withdrawals,199,199,1.00
9,Account Opening - MISC,196,196,1.00
